In [104]:
from sympy import (
    solve,
    Poly,
    Eq,
    Function,
    exp,
    Indexed,
    IndexedBase,
    Tuple,
    sqrt,
    Symbol,
    sin,
    limit,
    root,
    real_root,
    S,
    nroots,
)
from sympy.solvers.solvers import unrad
import numpy as np
import pandas as pd
from scipy.optimize import fsolve
from scipy.optimize import minimize

In [105]:
R = 41  # rate range 150 - 350 by 5s
H = 401  # hours index from 0.0 - 100.0 by 0.25s


In [106]:
df = pd.DataFrame((np.outer(hours, rate)))  # init df, labels
df.columns = [150 + (iter * 5) for iter in range(R)]
df.index = [iter * 0.25 for iter in range(H)]


In [107]:
df.head()

,150,155,160,165,170,175,180,185,190,195,...,305,310,315,320,325,330,335,340,345,350
0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00,...,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0
0.25,37.5,38.75,40.0,41.25,42.5,43.75,45.0,46.25,47.5,48.75,...,76.25,77.5,78.75,80.0,81.25,82.5,83.75,85.0,86.25,87.5
0.50,75.0,77.50,80.0,82.50,85.0,87.50,90.0,92.50,95.0,97.50,...,152.50,155.0,157.50,160.0,162.50,165.0,167.50,170.0,172.50,175.0
0.75,112.5,116.25,120.0,123.75,127.5,131.25,135.0,138.75,142.5,146.25,...,228.75,232.5,236.25,240.0,243.75,247.5,251.25,255.0,258.75,262.5
1.00,150.0,155.00,160.0,165.00,170.0,175.00,180.0,185.00,190.0,195.00,...,305.00,310.0,315.00,320.0,325.00,330.0,335.00,340.0,345.00,350.0


In [116]:
search = 150

In [121]:
mask = df.mask(df != search, False) # 

In [128]:
mask = mask[mask.any(axis=1)] # mask matching value
mask_id = mask.loc[:, (mask != 0).any(axis=0)]

mask_id


,150,200,300
0.50,False,False,150.0
0.75,False,150.0,False
1.00,150.0,False,False


In [148]:
mask[mask > 0].stack().index.tolist()

[(0.5, 300), (0.75, 200), (1.0, 150)]